In [2]:
import requests

from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import pandas as pd

In [3]:
llm = OpenAI(temperature=0)

### Joining

In [4]:
table_1 = pd.DataFrame(
    [
        {"id": "abc123", "name": "Alicia Alicestein", "Address": "123 Fake Street, 02421"},
        {"id": "abc124", "name": "Bob Bobua", "Address": "15 Elevenstein Road, 01515"},
    ]
)

table_2 = pd.DataFrame(
    [
        {"id": "def111", "name": "Bob B.", "zip code": "01515"},
        {"id": "def222", "name": "Alicia", "zip code": "01515"},
        {"id": "def333", "name": "Alicia", "zip code": "02421"},
    ]
)

In [5]:
prompt_template = """
You have two database tables. Your job is to match the records in each one.

Table 1:
{table_1_json}

Table 2:
{table_2_json}

Return a JSON array with matched IDs, e.g.,
[{{"table 1 id": ___, "table 2 id": ___}}], etc.

Do not return code or explain your reasoning. Just return the JSON result.
"""

In [6]:
prompt = PromptTemplate(
    input_variables=["table_1_json", "table_2_json"],
    template=prompt_template,
)

chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
table_1_json = table_1.to_json(orient="records", indent=2)
table_2_json = table_2.to_json(orient="records", indent=2)

print(chain.run(table_1_json=table_1_json, table_2_json=table_2_json))


[
  {"table 1 id": "abc123", "table 2 id": "def333"},
  {"table 1 id": "abc124", "table 2 id": "def111"}
]


Try with larger tables:

In [8]:
html = requests.get(
    "https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States",
    headers={'User-agent': 'Mozilla/5.0'}
).text
dfs = pd.read_html(html) # load with user agent to avoid 401 error

In [9]:
presidents_df = dfs[0].sample(25, random_state=0)

In [10]:
presidents_df.head()

,No.[a],Portrait,Name(Birth–Death),Term[14],Party[b][15],Party[b][15].1,Election,Vice President[16]
25,26,NaN,Theodore Roosevelt(1858–1919)[51],"September 14, 1901[p]–March 4, 1909",NaN,Republican,– 1904,"Vacant throughMarch 4, 1905 Charles W. Fairbanks"
22,23,NaN,Benjamin Harrison(1833–1901)[49],"March 4, 1889–March 4, 1893",NaN,Republican,1888,Levi P. Morton
28,29,NaN,Warren G. Harding(1865–1923)[54],"March 4, 1921–August 2, 1923[e]",NaN,Republican,1920,Calvin Coolidge
4,5,NaN,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",NaN,Democratic-Republican,1816 1820,Daniel D. Tompkins
10,11,NaN,James K. Polk(1795–1849)[34],"March 4, 1845–March 4, 1849",NaN,Democratic,1844,George M. Dallas


In [11]:
assert(presidents_df.index.is_unique)

table_2 = presidents_df.sample(20, random_state=0)

In [12]:
import re


def _parse_name(text: str) -> str:
    """Parse name name, birth year, and death year from Names(Birth-Death) column."""

    pattern = r"(.*)\([^0-9]*(\d{4})\)*"
    name, birth_year = re.search(pattern, text).groups()
    birth_year = int(birth_year)
    
    
    return name, birth_year


def _modify_name_pattern_1(name: str) -> str:
    """Modify 'First Last' to 'First L.'"""
    
    split = name.split()
    
    return f"{split[0]} {split[-1][0]}."


def _modify_name_pattern_2(name: str) -> str:
    """Modify 'First Last' to 'F. Last'"""
    
    split = name.split()
    
    return f"{split[0][0]}. {split[-1]}"

In [13]:
table_2["raw_name"], table_2["birth_year"] = zip(*table_2["Name(Birth–Death)"].apply(_parse_name))

table_2["name"] = ""
sample_idx = table_2.index.isin(table_2.sample(frac=0.5, random_state=1).index)
table_2.loc[sample_idx, "name"] = table_2.loc[sample_idx, "raw_name"].apply(_modify_name_pattern_1)
table_2.loc[~sample_idx, "name"] = table_2.loc[~sample_idx, "raw_name"].apply(_modify_name_pattern_2)

table_2 = table_2.rename(columns={"Party[b][15].1": "party"})

table_2 = table_2[["name", "birth_year", "party"]]

In [14]:
table_1 = presidents_df[["Name(Birth–Death)", "Term[14]", "Party[b][15].1", "Election", "Vice President[16]"]].copy()

In [15]:
def _assign_id(df: pd.DataFrame) -> pd.DataFrame:
    """Prepend id to columns."""

    columns = df.columns.to_list()
    df["id"] = df["id"] = range(df.shape[0])
    df = df[["id"] + columns]

    return df

In [16]:
table_1 = _assign_id(table_1)
table_2 = _assign_id(table_2)

In [17]:
table_1.head(3)

,id,Name(Birth–Death),Term[14],Party[b][15].1,Election,Vice President[16]
25,0,Theodore Roosevelt(1858–1919)[51],"September 14, 1901[p]–March 4, 1909",Republican,– 1904,"Vacant throughMarch 4, 1905 Charles W. Fairbanks"
22,1,Benjamin Harrison(1833–1901)[49],"March 4, 1889–March 4, 1893",Republican,1888,Levi P. Morton
28,2,Warren G. Harding(1865–1923)[54],"March 4, 1921–August 2, 1923[e]",Republican,1920,Calvin Coolidge


In [18]:
table_2.head(3)

,id,name,birth_year,party
31,0,F. Roosevelt,1882,Democratic
28,1,Warren H.,1865,Republican
41,2,Bill C.,1946,Democratic


In [19]:
ground_truth = table_1.join(table_2, lsuffix="_1", rsuffix="_2")

print(ground_truth.shape)

ground_truth.head()

(25, 10)


,id_1,Name(Birth–Death),Term[14],Party[b][15].1,Election,Vice President[16],id_2,name,birth_year,party
25,0,Theodore Roosevelt(1858–1919)[51],"September 14, 1901[p]–March 4, 1909",Republican,– 1904,"Vacant throughMarch 4, 1905 Charles W. Fairbanks",NaN,NaN,NaN,NaN
22,1,Benjamin Harrison(1833–1901)[49],"March 4, 1889–March 4, 1893",Republican,1888,Levi P. Morton,10.0,Benjamin H.,1833.0,Republican
28,2,Warren G. Harding(1865–1923)[54],"March 4, 1921–August 2, 1923[e]",Republican,1920,Calvin Coolidge,1.0,Warren H.,1865.0,Republican
4,3,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,NaN,NaN,NaN,NaN
10,4,James K. Polk(1795–1849)[34],"March 4, 1845–March 4, 1849",Democratic,1844,George M. Dallas,16.0,James P.,1795.0,Democratic


In [20]:
table_1_json = table_1.to_json(orient="records", indent=2)
table_2_json = table_2.to_json(orient="records", indent=2)

result = chain.run(table_1_json=table_1_json, table_2_json=table_2_json)

print(result)


[{"table 1 id": 0, "table 2 id": 0},{"table 1 id": 1, "table 2 id": 10},{"table 1 id": 2, "table 2 id": 1},{"table 1 id": 3, "table 2 id": 12},{"table 1 id": 4, "table 2 id": 16},{"table 1 id": 5, "table 2 id": 17},{"table 1 id": 6, "table 2 id": 14},{"table 1 id": 7, "table 2 id": 18},{"table 1 id": 8, "table 2 id": 13},{"table 1 id": 9, "table 2 id": 17},{"table 1 id": 10, "table 2 id": 11},{"table 1 id": 11, "table 2 id": 4},{"table 1 id": 12, "table 2 id": 19},{"table 1 id": 13, "table 2 id": 12},{"table 1 id": 14, "table 2 id": 9},{"table 1 id": 15, "table 2 id": 10},{"table 1 id": 16, "table 2 id": 3},{"table 1 id": 17, "table 2 id": 6},{"table 1 id": 18, "table 2 id": 15},{"table 1 id": 19,


In [21]:
def _hack_to_valid_json(invalid_json_str: str) -> str:
    """Turn an invalid JSON string to a valid one, sometimes."""
    
    return result[:result.rfind("}") + 1] + "]"

In [22]:
import json

predicted_ids = json.loads(_hack_to_valid_json(result))

predicted_ids[:3]

[{'table 1 id': 0, 'table 2 id': 0},
 {'table 1 id': 1, 'table 2 id': 10},
 {'table 1 id': 2, 'table 2 id': 1}]

In [23]:
predicted_id_pairs = pd.Series(tuple(pair.values()) for pair in predicted_ids)

In [24]:
gt = ground_truth.dropna(subset=["id_1", "id_2"])

ground_truth_id_pairs = pd.Series(zip(gt["id_1"], gt["id_2"]))

In [25]:
recall = ground_truth_id_pairs.isin(predicted_id_pairs).mean()
precision = predicted_id_pairs.isin(ground_truth_id_pairs).mean()

print(f"Recall: {recall}")
print(f"Precision: {precision}")

Recall: 0.7
Precision: 0.7368421052631579


### Flattening

In [26]:
html = requests.get(
    "https://www.sec.gov/Archives/edgar/data/320193/000119312510238044/d10k.htm", # Apple 10k filing from 2010
    headers={'User-agent': 'Mozilla/5.0'}
).text
dfs = pd.read_html(html) # load with user agent to avoid 401 error

table_csv = dfs[14].to_csv()

In [27]:
print(table_csv)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,,,,,,,,,,,
1,,,2010,2010.0,,,2009,2009.0,,,2008,2008.0,,,2007,2007.0,,,2006,2006.0,
2,Net sales,,$,65225.0,,,$,42905.0,,,$,37491.0,,,$,24578.0,,,$,19315.0,
3,Net income,,$,14013.0,,,$,8235.0,,,$,6119.0,,,$,3495.0,,,$,1989.0,
4,Earnings per common share:,,,,,,,,,,,,,,,,,,,,
5,Basic,,$,15.41,,,$,9.22,,,$,6.94,,,$,4.04,,,$,2.36,
6,Diluted,,$,15.15,,,$,9.08,,,$,6.78,,,$,3.93,,,$,2.27,
7,Cash dividends declared per common share,,$,0.0,,,$,0.0,,,$,0.0,,,$,0.0,,,$,0.0,
8,Shares used in computing earnings per share:,,,,,,,,,,,,,,,,,,,,
9,Basic,,,909461.0,,,,893016.0,,,,881592.0,,,,864595.0,,,,844058.0,
10,Diluted,,,924712.0,,,,907005.0,,,,902139.0,,,,889292.0,,,,877526.0,
11,,,,,,,,,,,,,,,,,,,,,
12,"Total cash, cash equivalents and marketable securities",,$,51011.0,,,$,33992.0,,,$,24490.0,,,$,15386.0,,,$,10110.0,
13,Total assets,,$,75183.0,,,$,47501.0,,,$,36171.0,,,$,24878.0,,,$,17205.0,
14,Total long-term obligations (a),,$,553

In [28]:
example_df = pd.DataFrame([{"Electric": "£128.13", "Water/Sewer": "£134.90"}], index=["10/6/2022"])

example_df

,Electric,Water/Sewer
10/6/2022,£128.13,£134.90


In [31]:
prompt = f"""
Flatten this table by inferring names for the columns and returning the output as JSON records.
Do not explain your reasoning or provide code. Just return the JSON records.

For example, if the input is

	Electric	Water/Sewer
10/6/2022	£128.13	£134.90

You would return
[{{"date": "2022-10-06", "utility": "Electric", "value": 128.13, "currency": "GBP"}},
{{"date": "2022-10-06", "utility": "Water/Sewer", "value": 134.90, "currency": "GBP"}}

Notice that you must infer what the column names and table values represent and give them keys in the JSON output.
You should parse numbers as JSON integers or floats.

Here is the input:

{table_csv}

Return the JSON output.
"""

In [32]:
print(llm(prompt))


[{"date": "2010", "metric": "Net sales", "value": 65225.0, "currency": "USD"},
{"date": "2010", "metric": "Net income", "value": 14013.0, "currency": "USD"},
{"date": "2010", "metric": "Earnings per common share: Basic", "value": 15.41, "currency": "USD"},
{"date": "2010", "metric": "Earnings per common share: Diluted", "value": 15.15, "currency": "USD"},
{"date": "2010", "metric": "Cash dividends declared per common share", "value": 0.0, "currency": "USD"},
{"date": "2010", "metric": "Shares used in computing earnings per share: Basic", "value": 909461.0, "currency": "USD"},
{"date": "2010", "metric": "Shares used in computing earnings per share: Diluted", "value": 924712.0, "currency": "USD"},
{"date": "2010", "metric": "Total cash, cash equivalents and marketable securities",


### Question Answering

In [33]:
product_df = pd.DataFrame(
    [
        {"name": "The Da Vinci Code", "type": "book", "price": 15, "quantity": 300, "rating": 4},
        {"name": "Jurassic Park", "type": "book", "price": 13, "quantity": 400, "rating": 4.5},
        {"name": "Jurassic Park", "type": "film", "price": 8, "quantity": 6, "rating": 5},
        {"name": "Matilda", "type": "book", "price": 5, "quantity": 80, "rating": 4},
    ],
)

product_df

,name,type,price,quantity,rating
0,The Da Vinci Code,book,15,300,4.0
1,Jurassic Park,book,13,400,4.5
2,Jurassic Park,film,8,6,5.0
3,Matilda,book,5,80,4.0


In [34]:
product_json = product_df.to_json(orient="records")

In [35]:
prompt_template = """
Below is a table in JSON form. You will answer a question using the information in the table:

{product_json}

The question is:

{question}

Return the answer. Do not explain your reasoning.
"""

In [36]:
prompt = PromptTemplate(
    input_variables=["product_json", "question"],
    template=prompt_template,
)

chain = LLMChain(llm=llm, prompt=prompt)

In [37]:
print(chain.run(product_json=product_json, question="What book has the highest price and lowest rating?"))


The Da Vinci Code


In [38]:
print(chain.run(product_json=product_json, question="What books have the lowest rating?"))


The Da Vinci Code and Matilda.


In [39]:
print(chain.run(product_json=product_json, question="What is the price of the book with the highest rating?"))


5


In [40]:
print(chain.run(product_json=product_json, question="What is the price of the product of type book with the highest rating?"))


13


In [41]:
prompt_template = """
Given a dataframe `df` with columns {columns_string}, write Pandas code that does the following:

{question}

Return the answer. Do not explain your reasoning.
"""

In [42]:
prompt = PromptTemplate(
    input_variables=["columns_string", "question"],
    template=prompt_template,
)

chain = LLMChain(llm=llm, prompt=prompt)

In [43]:
columns_string = '"' + '", "'.join(product_df.columns) + '"'

In [44]:
print(chain.run(columns_string=columns_string, question="What is the price of the product of type book with the highest rating?"))


```
df[df['type'] == 'book']['price'].max()
```


In [45]:
print(chain.run(columns_string=columns_string, question="What book has the highest price and lowest rating?"))


```
df[(df['price'] == df['price'].max()) & (df['rating'] == df['rating'].min())]['name']
```


In [46]:
print(chain.run(columns_string=columns_string, question="What books have the lowest rating?"))


```
df.sort_values(by='rating', ascending=True).head(1)
```


In [47]:
print(chain.run(columns_string=columns_string, question="What is the price of the book with the highest rating?"))


```
df[df['rating'] == df['rating'].max()]['price'].iloc[0]
```
